In [1]:
# Load in data
import pandas as pd
import numpy as np
import csv
import shap
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

/Users/gijssmeets/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# Display all row names
print(train.columns)
print(train.shape)
print(test.shape)

Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')
(165034, 14)
(110023, 13)


In [6]:
# Display the first 5 rows of the data
print(train.head(5))

   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0       0.00              2        1.0             0.0        181449.97   
1       0.00              2        1.0             1.0         49503.50   
2       0.00              2        1.0             0.0        184866.69   
3  148882.54              1        1.0             1.0         84560.88   
4       0.00              2        1.0             1.0         15068.83   

   Exited  
0       0  
1       0  
2       0  
3       0  
4 

In [45]:
# Make geography a categorical variable (1, 2, 3)
train['Geography'] = train['Geography'].astype('category')
test['Geography'] = test['Geography'].astype('category')

train['Geography'] = train['Geography'].cat.codes
test['Geography'] = test['Geography'].cat.codes

# make gender a categorical variable (1, 2, 3)
train['Gender'] = train['Gender'].astype('category').cat.codes
test['Gender'] = test['Gender'].astype('category').cat.codes



In [46]:
from sklearn.linear_model import LogisticRegression

X = train[train.columns.drop(['id', 'Exited', 'CustomerId', 'Surname'])]
y = train['Exited']

clf = LogisticRegression().fit(X,y)

X_test = test[test.columns.drop(['id', 'CustomerId', 'Surname'])]

predictions = clf.predict_proba(X_test)[:,1]

In [49]:
test_id = np.array(test['id'])
submission = np.column_stack((test_id, predictions))
np.savetxt("submissions/submission_improved.csv", submission, delimiter=",", header="id,Exited", comments='', fmt='%d,%f')